In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 1.2

In [48]:
# create a two-dimensional synthetic dataset of 300 points by sampling 100 points each from the three Gaussian distributions
np.random.seed(0)
datasets = []

# distrubution parameters
a_mean = [-1, -1]
a_cov = [[2, 0.5], [0.5, 1]]
b_mean = [1, -1]
b_cov = [[1, -0.5], [-0.5, 2]]
c_mean = [0, 1]
c_cov = [[1, 0], [0, 2]]

sigmas = [0.5, 1, 2, 4, 8]

for sigma in sigmas:
    pa = np.random.multivariate_normal(a_mean, sigma * np.array(a_cov), 100)
    pb = np.random.multivariate_normal(b_mean, sigma * np.array(b_cov), 100)
    pc = np.random.multivariate_normal(c_mean, sigma * np.array(c_cov), 100)

    data = np.concatenate((pa, pb, pc), axis=0)
    datasets.append(data)

In [49]:
# kmeans++ initialization
def kmeanspp(data, k, seed=0):
    np.random.seed(seed)
    centroids = np.zeros((k, data.shape[1]))
    # randomly choose the first centroid
    c1 = np.random.choice(data.shape[0], 1, replace=False)
    centroids[0] = data[c1]
    # choose the next k-1 centroids
    for i in range(1, k):
        D = np.zeros(data.shape[0])
        for j in range(data.shape[0]):
            D[j] = np.min(np.sum((data[j] - centroids[:i]) ** 2, axis=1))
        # randomly choose the next centroid with D probability
        centroids[i] = data[np.random.choice(data.shape[0], 1, p=D / np.sum(D))]

    return centroids


# evaluate clustering objective
def kmeans_clustering_objective(data, centroids, labels):
    obj = 0
    for i in range(data.shape[0]):
        label = int(labels[i])
        obj += np.sum((data[i] - centroids[label]) ** 2)
    return obj


# implement k-means
def kmeans(data, k, seed=0, max_iter=100, tol=1e-6):
    # initialize centroids using k-means++
    centroids = kmeanspp(data, k, seed)

    for iter in range(max_iter):
        new_centroids = np.zeros((k, data.shape[1]))
        # assign each point to the closest centroid
        labels = np.zeros(data.shape[0])
        for i in range(data.shape[0]):
            labels[i] = np.argmin(np.sum((data[i] - centroids) ** 2, axis=1))
        # update centroids
        for i in range(k):
            new_centroids[i] = np.mean(data[labels == i], axis=0)

        # check convergence
        if np.sum((new_centroids - centroids) ** 2) < tol:
            break

        centroids = new_centroids

    return centroids, labels

In [50]:
def GMM_clustering_objective(data, phi, mu, sigma):
    obj = 0
    for i in range(data.shape[0]):
        p = 0
        for j in range(len(phi)):
            p += (
                phi[j]
                * np.exp(
                    -0.5
                    * np.dot(
                        np.dot((data[i] - mu[j]).T, np.linalg.inv(sigma[j])),
                        (data[i] - mu[j]),
                    )
                )
                / np.sqrt(np.linalg.det(sigma[j]))
            )
        obj += -np.log(p)
    return obj

In [51]:
# implement GMM


def GMM(data, k, seed=0, max_iter=100, tol=1e-6):
    # initialize using k-means++
    centroids = kmeanspp(data, k, seed)
    labels = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        labels[i] = np.argmin(np.sum((data[i] - centroids) ** 2, axis=1))
    # initialize parameters
    phi = [np.sum(labels == i) / data.shape[0] for i in range(k)]
    mu = centroids
    sigma = [np.cov(data[labels == i].T) for i in range(k)]
    obj = GMM_clustering_objective(data, phi, mu, sigma)

    for iter in range(max_iter):
        # E-step: find the probability of each point belonging to each cluster
        gamma = np.zeros((data.shape[0], k))
        for i in range(data.shape[0]):
            for j in range(k):
                # joint probability of phi and z_i=j
                p_prior = phi[j]
                # conditional probability of x_i given z_i=j, which is given by the probability density function
                p_cond = np.exp(
                    -0.5
                    * np.dot(
                        np.dot((data[i] - mu[j]).T, np.linalg.inv(sigma[j])),
                        (data[i] - mu[j]),
                    )
                ) / np.sqrt(np.linalg.det(sigma[j]))
                gamma[i, j] = p_prior * p_cond
            # normalize gamma
            gamma[i] /= np.sum(gamma[i])

        # M-step: update parameters
        phi = np.sum(gamma, axis=0) / data.shape[0]
        mu = np.dot(gamma.T, data) / np.sum(gamma, axis=0).reshape(-1, 1)
        for j in range(k):
            sigma[j] = (
                np.dot((data - mu[j]).T, np.dot(np.diag(gamma[:, j]), (data - mu[j])))
                / np.sum(gamma, axis=0)[j]
            )

        # check convergence
        if GMM_clustering_objective(data, phi, mu, sigma) - obj < tol:
            break

        centroids = mu
        labels = np.argmax(gamma, axis=1)

    return centroids, labels, phi, mu, sigma

In [52]:
# evaluate both methods
evals = []

for i in range(len(sigmas)):
    sigma = sigmas[i]
    data = datasets[i]

    ### kmeans ###
    # run 5 times and choose the best result
    best_obj = np.inf
    best_acc = 0

    for seed in range(5):
        centroids, labels = kmeans(data, k=3, seed=seed, max_iter=100, tol=1e-6)

        # determine the cluster labels of the three Gaussian distributions
        a_label = np.argmin(np.sum((centroids - a_mean) ** 2, axis=1))
        b_label = np.argmin(np.sum((centroids - b_mean) ** 2, axis=1))
        c_label = np.argmin(np.sum((centroids - c_mean) ** 2, axis=1))

        obj = kmeans_clustering_objective(data, centroids, labels)

        # evaluate clustering accuracy
        # a_label: 1 b_label: 2 c_label: 0
        true_labels = np.concatenate(
            (np.ones(100) * a_label, np.ones(100) * b_label, np.ones(100) * c_label)
        )
        acc = np.sum(labels == true_labels) / data.shape[0]

        if obj < best_obj:
            best_obj = obj
            best_acc = acc

    evals.append([sigma, "kmeans", best_obj, best_acc])

    ### GMM ###
    # run 5 times and choose the best result
    best_obj = np.inf
    best_acc = 0

    for seed in range(5):
        centroids, labels, phi, mu, cov_sigma = GMM(
            data, k=3, seed=seed, max_iter=100, tol=1e-6
        )

        # determine the cluster labels of the three Gaussian distributions
        a_label = np.argmin(np.sum((centroids - a_mean) ** 2, axis=1))
        b_label = np.argmin(np.sum((centroids - b_mean) ** 2, axis=1))
        c_label = np.argmin(np.sum((centroids - c_mean) ** 2, axis=1))

        obj = GMM_clustering_objective(data, phi, mu, cov_sigma)

        # evaluate clustering accuracy
        # a_label: 1 b_label: 2 c_label: 0
        true_labels = np.concatenate(
            (np.ones(100) * a_label, np.ones(100) * b_label, np.ones(100) * c_label)
        )
        acc = np.sum(labels == true_labels) / data.shape[0]

        if obj < best_obj:
            best_obj = obj
            best_acc = acc

    evals.append([sigma, "GMM", best_obj, best_acc])

evals = pd.DataFrame(evals, columns=["sigma", "method", "objective", "accuracy"])
evals

,sigma,method,objective,accuracy
0,0.5,kmeans,323.349584,0.800000
1,0.5,GMM,375.244801,0.766667
2,1.0,kmeans,509.481965,0.736667
3,1.0,GMM,508.618881,0.690000
4,2.0,kmeans,856.701568,0.623333
5,2.0,GMM,672.038896,0.563333
6,4.0,kmeans,1658.608710,0.556667
7,4.0,GMM,848.668799,0.630000
8,8.0,kmeans,2931.046595,0.496667
9,8.0,GMM,1022.159053,0.596667


In [63]:
# plot evals
evals["sigma"] = evals["sigma"].astype(str)
fig = px.bar(evals, x="method", y="objective", color="sigma", barmode="group")
fig.update_layout(height=500, width=600)
fig.show()

fig = px.bar(evals, x="sigma", y="accuracy", color="method", barmode="group")
fig.update_layout(height=500, width=600)
fig.show()

# 2.3

In [132]:
# implement PCA & demeaned PCA
def PCA(X, d, preprocess=None):
    # X: n * D
    if preprocess == "demean":
        # demean X
        X_ = X - np.mean(X, axis=0)
    elif preprocess == "normalize":
        # normalize X
        X_ = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    else:
        X_ = X
    # svd
    U, S, Vt = np.linalg.svd(X_)
    # choose singular values and vectors
    S_d = np.diag(S[:d])
    Vt_d = Vt[:d, :]
    # get the scores
    Z = U[:, :d] @ S_d
    # Compute the reconstructions in D dimensions
    X_re = Z @ Vt_d
    # reverse
    if preprocess == "demean":
        X_re = X_re + np.mean(X, axis=0)
    elif preprocess == "normalize":
        X_re = X_re * np.std(X, axis=0) + np.mean(X, axis=0)
    return Z, S_d, Vt_d, X_re

In [369]:
# implement DRO
def DRO(X, d):
    n = X.shape[0]
    coef = np.eye(n) - np.ones([n, 1]) @ np.ones([1, n]) / n

    # get b
    b = X.T @ np.ones([n, 1]) / n
    Y = coef @ X

    # low rank approximation
    U, S, Vt = np.linalg.svd(Y)
    S_d = np.diag(S[:d])
    Vt_d = Vt[:d, :]

    # compute the reconstructions
    Y_d = U[:, :d] @ S_d @ Vt_d

    # restore
    X_res = Y_d + np.ones([n, 1]) @ b.T
    Z = U[:, :d] @ S_d
    A = Vt_d.T
    
    return Z, S_d, A, X_res

In [370]:
# read data
data2D = pd.read_csv(
    r"C:\Users\Zheng\OneDrive - UW-Madison\PhD project\Coursework\CS760\hw5\data\data2D.csv",
    header=None,
).to_numpy()
data1000D = pd.read_csv(
    r"C:\Users\Zheng\OneDrive - UW-Madison\PhD project\Coursework\CS760\hw5\data\data1000D.csv",
    header=None,
).to_numpy()

In [381]:
# choose d for data1000D
X = data1000D

n = X.shape[0]
coef = np.eye(n) - np.ones([n, 1]) @ np.ones([1, n]) / n

# get b
Y = coef @ X

# low rank approximation
U, S, Vt = np.linalg.svd(Y)
    
# plot the singular values
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, 100), y=S))
fig.update_layout(height=500, width=600)

# annotate the elbow point
for i in [30, 31]:
    fig.add_annotation(
        x=i,
        y=S[i - 1],
        text="d=" + str(i) + ", S=" + str(round(S[i - 1], 2)),
        showarrow=True,
        font=dict(size=16, color="black"),
    )
fig.show()


In [373]:
# get dimensionality reduction results
Zs = []
S_ds = []
Vt_ds = []
X_res = []

Z, S_d, Vt_d, X_re = PCA(data2D, d=1)
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)

Z, S_d, Vt_d, X_re = PCA(data2D, d=1, preprocess="demean")
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)

Z, S_d, Vt_d, X_re = PCA(data2D, d=1, preprocess="normalize")
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)


Z, S_d, Vt_d, X_re = DRO(data2D, d=1)
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)

In [374]:
# plot data for data2D
methods = ["buggy PCA", "demeaned PCA", "normalized PCA", "DRO"]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=data2D[:, 0],
        y=data2D[:, 1],
        mode="markers",
        name="original",
    )
)


for i in range(len(methods)):
    fig.add_trace(
        go.Scatter(
            x=X_res[i][:, 0],
            y=X_res[i][:, 1],
            mode="markers",
            name=methods[i],
        )
    )

fig.update_layout(height=500, width=600)
fig.show()

In [375]:
# reconstruction error
errors = []
for i in range(len(methods)):
    errors.append(np.sum((data2D - X_res[i]) ** 2))

errors = pd.DataFrame(errors, columns=["error"])
errors["method"] = methods
errors

,error,method
0,44.345154,buggy PCA
1,0.500304,demeaned PCA
2,2.473604,normalized PCA
3,0.500304,DRO


In [387]:
# get dimensionality reduction results
Zs = []
S_ds = []
Vt_ds = []
X_res = []

Z, S_d, Vt_d, X_re = PCA(data1000D, d=30)
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)

Z, S_d, Vt_d, X_re = PCA(data1000D, d=30, preprocess="demean")
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)

Z, S_d, Vt_d, X_re = PCA(data1000D, d=30, preprocess="normalize")
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)


Z, S_d, Vt_d, X_re = DRO(data1000D, d=30)
Zs.append(Z)
S_ds.append(S_d)
Vt_ds.append(Vt_d)
X_res.append(X_re)

In [388]:
# reconstruction error
errors = []
for i in range(len(methods)):
    errors.append(np.sum((data1000D - X_res[i]) ** 2))

errors = pd.DataFrame(errors, columns=["error"])
errors["method"] = methods
errors

,error,method
0,401365.699310,buggy PCA
1,136522.979489,demeaned PCA
2,136814.290499,normalized PCA
3,136522.979489,DRO
